In [2]:
import pandas as pd
import numpy as np
import os

In [4]:
folder_path = '../../Data/sclc_ucologne_2015'
file_name = 'data_clinical_patient.txt'
file_path = folder_path + '/' + file_name
# Load the RNAseq data from the csv file
data = pd.read_csv(file_path, sep='\t', index_col=0)

# Get the column names
data.head()
# the columns correspond to the patient IDs

,Diagnosis Age,Sex,Ethnicity Category,N Stage,M Stage,UICC Tumor Stage,Smoker,Smoking History,Previous Treatment,First Pathologic Diagnosis Biospecimen Acquisition Method Type,Neoadjuvant Chemotherapy,Chemotherapy,Radiation Therapy,Overall Survival Status,Progress Free Survival (Months),Overall Survival (Months)
#Patient Identifier,,,,,,,,,,,,,,,,
#Identifier to uniquely specify a patient.,Age at which a condition or disease was first ...,Sex,The text for reporting information about ethni...,N Stage,M Stage,UICC Tumor Stage,Indicates if person is a smoker.,Smoking History,Previous treatment.,Text name of the procedure to secure the tissu...,Neoadjuvant Chemotherapy.,Chemotherapy.,Radiation Therapy.,Overall patient survival status.,Progress Free Survival (Months),Overall survival in months since initial diago...
#STRING,NUMBER,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER,NUMBER
#1,1,1,1,1,1,1,1,1,1,1,1,1,1,9,1,1
PATIENT_ID,AGE,SEX,ETHNICITY,N_STAGE,M_STAGE,UICC_TUMOR_STAGE,SMOKER,SMOKING_HISTORY,PREVIOUS_TREATMENT,METHOD_OF_SAMPLE_PROCUREMENT,NEOADJ_CHEMO,CHEMOTHERAPY,RADIATION_THERAPY,OS_STATUS,PFS_MONTHS,OS_MONTHS
sclc_ucologne_2015_S00022,47.0,Male,NaN,2,0,IIIa,Current,NaN,Untreated,Surgical Resection,No,Yes,Yes,1:DECEASED,36.0,38.0
